In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

import sys
sys.path.append('..')
import numpy as np
from src.game import Board, Game
from src.policy_value_net_tensorflow import PolicyValueNet
from src.mcts_pure import MCTSPlayer as MCTS_Pure
from src.mcts_alphaZero import MCTSPlayer
from src import results_dir

In [ ]:
n = 4
width, height = 6, 6
start_player = 1

model_file = os.path.join(results_dir, "zero_17_4_15:36", "policy_1500.model")

board = Board(width=width, height=height, n_in_row=n)
game = Game(board)

best_policy = PolicyValueNet(width, height, model_file=model_file)
ai = MCTSPlayer(best_policy.policy_value_fn, c_puct=5, n_playout=400)

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
ax.set_xlim([0, 5])
ax.set_ylim([0, 5])
ax.grid(c='k')
ax.tick_params(axis='both', which='major', pad=15)

fig.canvas.draw()
board.init_board(start_player)

def tellme(s):
    ax.set_title(s, fontsize=16, y=1.05)
    fig.canvas.draw()

def onclick(event):    
    y = int(np.round(event.xdata))
    x = int(np.round(event.ydata))
    
    ax.plot(y, x, 'o', ms=30, color='k', mec='k', zorder=100, clip_on=False)
    fig.canvas.draw()
    
    x_ = (5 - x)
    move = board.location_to_move((x_, y))
    if move not in board.availables:
        tellme("Invalid move, try again..")
        return

    board.do_move(move)
    end, winner = board.game_end()
    if end:
        if winner != -1:
            tellme("Game over. You won!")
        else:
            tellme("Game end. Tie")
    else:
        tellme("The AI is thinking...")

        move = ai.get_action(board)
        board.do_move(move)

        x, y = board.move_to_location(move)
        x = (5 - x)
        ax.plot(y, x, 'o', ms=30, color='w', mec='k', zorder=100, clip_on=False)

        tellme("The AI's last move: {}, {}, you turn".format(y,x))

        end, winner = board.game_end()
        if end:
            if winner != -1:
                tellme("Game over. The AI won")
            else:
                tellme("Game over. Tie")

if start_player == 1:
    tellme("The AI is thinking...")
    move = ai.get_action(board)
    board.do_move(move)
    x, y = board.move_to_location(move)
    x = 5 - x
    ax.plot(y, x, 'o', ms=30, color='w', mec='k')

    tellme("AI's last move: {}, {}, you turn".format(y,x))
    
cid = fig.canvas.mpl_connect('button_press_event', onclick)